In [1]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "gurobi"],  # modules to install
    license_uuid="your-license-uuid",  # license to use
)  # instantiate AMPL object and register magics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.0 MB/s eta 0:00:00


In [77]:
%%ampl_eval
reset;


set KONCENTRATY;

# ograniczenia wynikające z natury zmiennych (np. masa i cena nie moga byc ujemne)
param bialko {KONCENTRATY} >= 0;
param sole{KONCENTRATY} >= 0;
param witaminy{KONCENTRATY} >= 0;
param energia{KONCENTRATY} >= 0;
param cena{KONCENTRATY} >= 0;
param min_bialko >= 0;
param min_sole >= 0;
param min_witaminy >= 0;
param max_cena > 0;
var masa {KONCENTRATY} >= 0;

var wartosc_calkowita = sum { k in KONCENTRATY } masa[k]*cena[k];

# ograniczenia wynikające wprost z treści zadania
subject to calk_bialko: sum { k in KONCENTRATY } masa[k]*bialko[k] >= min_bialko;
subject to calk_sole: sum { k in KONCENTRATY } masa[k]*sole[k] >= min_sole;
subject to calk_witaminy: sum { k in KONCENTRATY } masa[k]*witaminy[k] >= min_witaminy;
subject to wart_calk_ogr: wartosc_calkowita <= max_cena;

#zmienne stanu
var W = sum { k in KONCENTRATY } masa[k]*energia[k];
var O = (max_cena - wartosc_calkowita);

#wagi w1, w2 do funkcji skalaryzujących
param w1 >= 0 <= 1;
param w2 := 1 - w1;

#funkcja celu ze skalaryzacja srednia wazona
maximize Z: w1*W + w2*O;

#funkcja celu ze skalaryzacja maksyminowa
maximize Z: min(w1*W, w2*O);

In [86]:
%%ampl_eval
reset data;
data;

set KONCENTRATY := K1 K2;

param: bialko :=
 K1 60
 K2 50
  ;

param: sole :=
 K1 24
 K2 60
  ;

param: witaminy :=
 K1 0
 K2 10
  ;

param: energia :=
 K1 400
 K2 200
  ;

param: cena :=
 K1 300
 K2 250
  ;

param min_bialko  := 150;
param min_sole  := 120;
param min_witaminy  := 10;
param max_cena := 1200;

param w1 := 0.45;

In [87]:
%%ampl_eval
option solver highs;
solve;

display masa;
display O;
display W;

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 660
0 simplex iterations
0 barrier iterations
masa [*] :=
K1  3.16667
K2  1
;

O = 0

W = 1466.67



[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)